In [1]:
import warnings

warnings.filterwarnings("ignore")

from typing import List, Optional, Union

import numpy as np
import pandas as pd

from tsururu.dataset import IndexSlicer, Pipeline, TSDataset
from tsururu.models import (
    CatBoost, 
    MeanBlender, 
    BestModel, 
    ClassicBlender
)
from tsururu.strategies import (
    DirectStrategy,
    FlatWideMIMOStrategy,
    MIMOStrategy,
    RecursiveStrategy,
)
from tsururu.transformers import (
    DateSeasonsGenerator,
    DifferenceNormalizer,
    LagTransformer,
    LastKnownNormalizer,
    SequentialTransformer,
    StandardScalerTransformer,
    TargetGenerator,
    UnionTransformer,
)
from tsururu.model_training.trainer import MLTrainer
from tsururu.model_training.validator import KFoldCrossValidator

In [2]:
def get_results(
    cv: int,
    regime: str,
    y_true: Optional[List[np.ndarray]] = None,
    y_pred: Optional[List[np.ndarray]] = None,
    ids: Optional[List[Union[float, str]]] = None,
) -> pd.DataFrame:
    def _get_fold_value(
        value: Optional[Union[float, np.ndarray]], idx: int
    ) -> List[Optional[Union[float, np.ndarray]]]:
        if value is None:
            return [None]
        if isinstance(value[idx], float):
            return value[idx]
        if isinstance(value[idx], np.ndarray):
            return value[idx].reshape(-1)
        raise TypeError(f"Unexpected value type. Value: {value}")

    df_res_dict = {}

    for idx_fold in range(cv):
        # Fill df_res_dict
        for name, value in [("y_true", y_true), ("y_pred", y_pred)]:
            df_res_dict[f"{name}_{idx_fold+1}"] = _get_fold_value(
                value, idx_fold
            )
        if regime != "local":
            df_res_dict[f"id_{idx_fold+1}"] = _get_fold_value(ids, idx_fold)

    # Save datasets to specified directory
    df_res = pd.DataFrame(df_res_dict)
    return df_res

There are several main objects to look out for when working with the library:
1) `TSDataset`.
2) `Pipeline` and `Transformers`
3) `Strategy`.
4) `Model`.

### TSDataset

This class is needed to store data and meta-information about it.

To initialise it is necessary to submit the data in `pd.DataFrame` format and define some meta-information about roles that necessary for solving the task of time series forecasting: `id`, `date`, `target`.

In [3]:
df_path = "datasets/global/train_kaggle.csv"
train_df = pd.read_csv(df_path, sep=",")
train_df.sample(5)

,Store_id,SKU_id,Date,Promo,Demand,Regular_Price,Promo_Price
11286,23,1,21.04.2015,NaN,94,138.50,NaN
21302,42,1,19.04.2016,1.0,1414,135.78,122.32
54850,22,2,25.03.2016,NaN,0,138.36,NaN
21660,43,1,21.11.2015,NaN,412,130.34,NaN
71412,55,2,05.09.2015,NaN,13,128.17,NaN


In [4]:
train_df["Date"] = pd.to_datetime(train_df['Date'], dayfirst=True, format="mixed")
train_df["id"] = train_df["Store_id"].astype("str") + "_" + train_df["SKU_id"].astype("str") 

In [5]:
dataset_params = {
    "target": {
        "columns": ["Demand"],
        "type": "continious",
    },
    "date": {
        "columns": ["Date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [6]:
dataset = TSDataset(
    data=train_df,
    columns_params=dataset_params,
    print_freq_period_info=True,
)

freq: Day; period: 1


### Pipeline and Transformers

#### What kind of transformers are there?

Special attention should be paid to the `Transformer` class: the elements of the pipeline that are responsible for transforming the values of a series and generating features. `Pipeline` class is a wrapper over transformers which is needed to provide some additional methods and functions above them.

There are two types of transformers that are used to collect pipelines:
- `Union` transformers;
- `Sequential` transformers.

Below is a list of available Transformers: 
- `StandardScalerTransformer` *(Series4Series)*.
- `DifferenceNormalizer` *(Series4Series)*: subtract the previous value or divide by it.
- `LastKnownNormalizer` *(Features4Features)*: normalize all lags by the last known one: divide by it or subtract.

This three transformers provide flags `transform_features` / `transform_target`, that allow you to manipulate traits and targets separately and get different results from them.

Besides, __DifferenceNormalizer__ and __LastKnownNormalizer__ can be applied in two regimes: `delta` and `ratio`: in the first case, normalisation means subtracting the target value from the current value, and in the second, dividing by it.

- `LabelEncodingTransformer` and `OneHotEncodingTransformer` *(Series4Series)* - encoders for categorical features.
- `TimeToNumGenerator` and `DateSeasonsGenerator` *(Series4Series)* - generator for seasonal features by dates.
- `LagTransformer` *(Series4Features) - generator for lags. 

__!!!The lag transformer must necessarily be present in the sequential transformer, otherwise the features will not be generated.!!!__

Finally, to generate targets, you need to use `TargetGenerator`.

#### Transformers must be assembled in order!

The __SeriesToSeries__ transformers should come first, followed by the LagTransformer and TargetGenerator (__SeriesToFeatures__), and then the __FeaturesToFeatures__ transformers.

!!!Thus, `StandardScalerNormalizer` and `DifferenceNormalizer` should be before `LagTransformer` and `LastKnownNormalizer` after it!!!

#### How to build a Pipeline?

So, there are two ways to build a pipline from transformers: initialise the transformers of interest by hand or use a config in the form of a dictionary. Let's look at both ways.

In [7]:
standard_scaler = StandardScalerTransformer(
    transform_features=True,
    transform_target=True
)

lag = LagTransformer(lags=3)
date_lag = LagTransformer(lags=3)
id_lag = LagTransformer(lags=1)

target_generator = TargetGenerator()

date_seasons = DateSeasonsGenerator(
    seasonalities=["doy", "m", "wd"],
    from_target_date=True,
)

In [8]:
union_1 = UnionTransformer(transformers_list=[lag, target_generator])

seq_1 = SequentialTransformer(transformers_list=[standard_scaler, union_1], input_features=["Demand"])
seq_2 = SequentialTransformer(transformers_list=[date_seasons, date_lag], input_features=["Date"])
seq_3 = SequentialTransformer(transformers_list=[id_lag], input_features=["id"])

union = UnionTransformer(transformers_list=[seq_1, seq_2, seq_3])

In [9]:
pipeline_1 = Pipeline(union, multivariate=False)

In [10]:
pipeline_1.__dict__

{'transformers': <tsururu.transformers.base.UnionTransformer at 0x1f2a2fe99d0>,
 'multivariate': False,
 'is_fitted': False,
 'strategy_name': None,
 'output_features': None,
 'y_original_shape': None}

Or:

In [11]:
pipeline_params = {
    "target": {
        "columns": ["Demand"],
        "features": {
            "StandardScalerTransformer":
                {
                    "transform_target": True, 
                    "transform_features": True
                },
            "LagTransformer": {"lags": 7},
        },
    },
    "date": {
        "columns": ["Date"],
        "features": {
            "DateSeasonsGenerator": {
                # Use seasonality features from the date column as 
                # features with datetime lags
                # Possible values: [
                #    "y": year, "m": month, "d": day, 
                #    "wd": weekday, "doy": dayofyear,
                #    "hour": hour, "min": minute, "sec": second, 
                #    "ms": microsecond,  "ns": nanosecond
                # ]
                "seasonalities": ['doy', 'm', 'wd'], 
                # Use date from target point to make datetime features
                "from_target_date": True,
            },
            "LagTransformer": {"lags": 3}
        },
    },
    "id": {
        "columns": ["id"],
        "features": {
            "LabelEncodingTransformer": {},
            "LagTransformer": {"lags": 1},
        },
    }
}

In [12]:
pipeline = Pipeline.from_dict(pipeline_params, multivariate=False)

Or:

In [13]:
pipeline_easy_params = {
    # One from ["none", "standard_scaler", "difference_normalizer", "last_known_normalizer"]
    "target_normalizer": "standard_scaler",

    # One from ["none", "delta", "ratio"]  (MUST BE "none" for "standard_scaler" and NOT "none" for others)
    "normalizer_regime": "none",

    # One from ["features", "target", "features_target"]
    "normalizer_transform_regime": "features_target",

    "target_lags": 3,
    "date_lags": 1,
#    "exog_lags": 1,
}

In [14]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

#### Can I use exogenous variables in the pipeline?

Yes! Exogenous variables can also be specified here. Just include them in your pipeline.

However, their operation is currently tested only for the `MIMOStrategy` in global-modelling. For other strategies support of additional variables is under development.

In [15]:
# pipeline_params["exog_group_1"] = {
#     "columns": ["value"],
#     "features": {
#         "StandardScalerTransformer":
#             {
#                 "transform_target": False, 
#                 "transform_features": True
#             },
#         "LagTransformer": {"lags": 7},
#     },
# }

__Make sure you have the transform_target = False flag for exogenous features!__

#### Model

The model is separate from the strategy. Any model can be run in any strategy if it supports this input and output format

One of the easiest options – is to use GBM.

In [16]:
# Configure the model parameters
model_params = {
    "loss_function": "MultiRMSE",
    "early_stopping_rounds": 100,
    "verbose": 500,
}

# Configure the validation parameters
validation_params = {
     "n_splits": 2,
     "shuffle": True,
     "random_state": 42,
 }

In [17]:
model = CatBoost
validation = KFoldCrossValidator

In [18]:
trainer = MLTrainer(
    model, 
    model_params, 
    validation,
    validation_params,
)

#### Strategy

- _Recursive:_ 
    - one model for all points of the forecast horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: a prediction is iteratively made one point ahead, and then this prediction is used to further shape the features in the test data. 
- _Recursive-reduced:_
    - one model for all points in the prediction horizon;
    - *training*: the model is trained to predict one point ahead;
    - *prediction*: features are generated for all test observations at once, unavailable values are replaced by NaN.
- _Direct:_ 
    - individual models for each point in the prediction horizon. 
- _MultiOutput (MIMO - Multi-input-multi-output):_
    - one model that learns to predict the entire prediction horizon. 
    - __Also, this strategy supports the presence of `exogenous features` (only for local- or global-modelling).__
- _FlatWideMIMO:_.
    - mixture of Direct and MIMO, fit one model, but uses deployed over horizon Direct's features.
    - __Number of `lags for datetime features` should be equal to `horizon` while using this strategy.__

In [19]:
horizon = 7
model_horizon = 7
history = 7
step = 1

strategy = RecursiveStrategy(
    pipeline=pipeline,
    trainer=trainer,
    horizon=horizon,
    history=history,
    step=step,
)

In [20]:
fit_time, _ = strategy.fit(dataset)

ValueError: not enough values to unpack (expected 5, got 2)

In [ ]:
forecast_time, current_pred = strategy.predict(dataset)

In [ ]:
current_pred

## Backtest validation of pipeline

In [ ]:
ids, test, pred, fit_time, forecast_time = strategy.back_test(dataset, cv=1)

In [ ]:
get_results(cv=1, regime="global", y_true=test, y_pred=pred, ids=ids)

## Working with raw time series' granularity

Time series come in different granularities, from hourly and daily time series to more complex ones such as the end of each quarter.

If the rows do not contain segments that are too short (that are shorter than history + horizon), then `tsururu` will try to extract the row granularity on its own. We currently support the following types:

- Yearly (and YearlyEnd)
- Quarterly (and Quarterly)
- Monthly (and MonthlyEnd)
- Weekly
- Daily
- Hourly
- Minlutely
- Secondly
- Microsecondly

There is also support for compound granularities (10 days, 15 minutes, 32 seconds, etc.). The correctness of the selected granularity can be checked from the output after the `Dataset` class has been created.

However, there are tricky situations (e.g. 28 days) where the monthly granularity may be guessed incorrectly. Therefore, it is possible to set your own granularity using the `pd.DateOffset` class or related classes from `pandas.tseries.offsets`, which must be fed as `delta` parameter into the `Dataset` class. Then the time column will be processed according to the user's settings.

Consider a time series where each point is exactly __28 daily points away__ from each other

In [ ]:
df_path_2 = "datasets/global/simulated_data_to_check_28D.csv"

# Configure the features settings
dataset_params_2 = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

In [ ]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    print_freq_period_info=True,
)

We see that the frequency of the series is incorrectly defined as monthly. Let's try to pass the `delta` parameter.

In [ ]:
dataset_2 = TSDataset(
    data=pd.read_csv(df_path_2),
    columns_params=dataset_params_2,
    delta=pd.DateOffset(days=28),
    print_freq_period_info=True,
)

Now it's all detected correctly.